In [1]:
import numpy as np

In [2]:
class phi:
    def __init__(self,tpm,states=None):
        self.tpm = tpm # tpm will be size 2^n X 2^n
        self.num_nodes = int(np.log2(self.tpm.shape[0]))
        if (states is None):
            self.states = np.zeros((self.num_nodes))
        else:
            self.states = states
        
        self.nodes = np.arange(self.num_nodes)
    
    def state_to_row(self,states):
        decimal = 0
        for i,state in enumerate(states):
            decimal += (2**i) * (state)
        return decimal
    
    def row_to_state(self,row_number,num_states):
        binary = bin(row_number)[2:]
        states = []
        for state in binary: # convert row to binary
            states.append(int(state))
        
        states = np.flip(states) # flip to make little endian
       
        if (len(states) < num_states): 
            states = np.concatenate((states,np.zeros((num_states-len(states)),dtype=int)))
            
        return states
    
    
    def marginalize(self,tpm,candidate_nodes,background_nodes):
        sub_tpm_new = np.zeros((2**len(candidate_nodes),2**len(candidate_nodes)))
        
        # 2. Second, we marginalize over the column states, which means we sum the probabilities of columns which
        # only differ in the state of the background node.
        for j in range(2**len(candidate_nodes)):
            sub_tpm_new[:,j] = sub_tpm[:,j]
            for b_node in background_nodes:
                sub_tpm_new[:,j] += sub_tpm[:,j+2**b_node]
    
    """
    The candidate subsystem tpm gets generated in two parts:
    1. First, the t-1 states which contain background nodes in states that are not correct are eliminated. We say
        we're conditioning on the states of the background elements because we only look at the part of the transition
        probability matrix that involves the background conditions in their current state.
    2. Second, we marginalize over the column states, which means we sum the probabilities of columns which
        only differ in the state of the background node. 
    """
    def candidate_subsystem_tpm(self,candidate_nodes):
        background_nodes = np.setdiff1d(self.nodes,candidate_nodes)
        sub_tpm = None
        
        # 1. First, the t-1 states which contain background nodes in states that are not correct are eliminated
        for i,row in enumerate(self.tpm):
            include_row = True
            for b_node in background_nodes:
                # don't include a row in which a background node isn't in original state
                if (self.row_to_state(i,self.num_nodes)[b_node] != self.states[b_node]):
                    include_row = False
            
            if (include_row and sub_tpm is None):
                sub_tpm = self.tpm[i].reshape((1,-1))
            elif (include_row):
                sub_tpm = np.concatenate((sub_tpm,self.tpm[i].reshape((1,-1))),axis=0)
            
        
        sub_tpm_new = np.zeros((2**len(candidate_nodes),2**len(candidate_nodes)))
        
        # 2. Second, we marginalize over the column states, which means we sum the probabilities of columns which
        # only differ in the state of the background node.
        for j in range(2**len(candidate_nodes)):
            sub_tpm_new[:,j] = sub_tpm[:,j]
            for b_node in background_nodes:
                sub_tpm_new[:,j] += sub_tpm[:,j+2**b_node]
        
        return sub_tpm_new

In [3]:
test_tpm = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                    [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
                    [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                    [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
                    [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
                    [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]], dtype=int)
test = phi(test_tpm,[0,0,0,0])

In [4]:
test.num_nodes

4

In [13]:
test.row_to_state(5,4)

array([1, 0, 1, 0])

In [9]:
test.candidate_subsystem_tpm([1,2,3])

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])